This Python notebook is used for calculating the the Positron Emission Tomography (PET) and the Computed Tomography (CT) statistics such as Hounsfield Unit (HU) mean, Hounsfield Unit Standard Deviation, Standardized Uptake Value (SUV) mean, SUV peak, and SUV max. It also computes the volume of the organs of interest in ml.
1. Import the necessary libraries
2. Input files that are required for statistics computation. 
((i) DICOM file PET
(i) DICOM file CT
(iii) NIfTI file PET
(iv) NIfTI file CT
(v) Segmented organs)
3. Extract the requird metadata from the DICOM and store it. (Explanation: We are performing all of our calculations on NIfTI file and since NIfTI file does not have our necessary metadata in them, we need to extract the required metadata from the DICOM and use it for our calculations with NIfTI file)
4. Main functions
5. Output storage path (Excel file path to store the output)
6. Main code

1. Import the necessary libraries

In [ ]:
!pip install pydicom nibabel pandas matplotlib scipy openpyxl

In [ ]:
!pip install openpyxl

In [124]:
import pydicom
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import datetime
from datetime import timedelta
from scipy.ndimage import median_filter

2. Required input files path: (You would need to provide any one of the .dcm file just for extracting the necessary metadata from them. Specify the folder where all your 117 segmented organ classes are stored).

In [125]:
#DICOM file paths
pet_dicom_file_path = "C:/Personal/ABX/patient_1/DICOM/PETCT_0117d7f11f/09-13-2001-NA-PET-CT Ganzkoerper  primaer mit KM-68547/12.000000-PET corr.-44501/6-075.dcm"
ct_dicom_file_path = "C:/Personal/ABX/patient_1/DICOM/PETCT_0117d7f11f/09-13-2001-NA-PET-CT Ganzkoerper  primaer mit KM-68547/5.000000-GK p.v.2-84770/1-001.dcm"

# NIfTI file paths
nifti_file_path_pet = "C:/Personal/ABX/patient_1/NIFTI/PETCT_0117d7f11f/09-13-2001-NA-PET-CT Ganzkoerper  primaer mit KM-68547/PET.nii.gz"
nifti_file_path_ct = "C:/Personal/ABX/patient_1/NIFTI/PETCT_0117d7f11f/09-13-2001-NA-PET-CT Ganzkoerper  primaer mit KM-68547/CTres.nii.gz"

# Segmented organs pathh
segmented_organs_folder = "C:/Personal/ABX/patient_1/segmentations"

#For SUR
aorta_segmented_nifti_path = os.path.join(segmented_organs_folder, "aorta.nii.gz")

# Normalize the path to use forward slashes
aorta_segmented_nifti_path = aorta_segmented_nifti_path.replace('\\', '/')
print(aorta_segmented_nifti_path)

# Path to the aorta segmented NIfTI file
aorta_segmented_nifti_path = os.path.join(segmented_organs_folder, "aorta.nii.gz").replace('\\', '/')
# Load the aorta segmented NIfTI file
aorta_segmented_img = nib.load(aorta_segmented_nifti_path)
aorta_segmented_data = aorta_segmented_img.get_fdata()

# Load PET NIfTI file for aorta activity calculation
pet_img_for_aorta = nib.load(nifti_file_path_pet)
pet_data_for_aorta = pet_img_for_aorta.get_fdata()

# Calculate the mean activity within the aorta's segmentation
aorta_mean_activity = np.mean(pet_data_for_aorta[aorta_segmented_data > 0])

aorta_activity = aorta_mean_activity

print(aorta_activity)

C:/Personal/ABX/patient_1/segmentations_sample/aorta.nii.gz
7084.040157022099


3. Extract the required metadata from the DICOM and store it in an array named "metadata"

In [126]:
def get_data_element_value(data_element):
    # If the element is None, return 'Attribute not found'
    if data_element is None:
        return 'Attribute not found'
    # If the element is a pydicom DataElement, return its value
    elif isinstance(data_element, pydicom.dataelem.DataElement):
        return data_element.value
    # If the element is already a value (e.g., a string), return it directly
    else:
        return data_element
    
def dicom_time_to_timedelta(dicom_time_str):
    #Time is in HHMMSS.FFFFFF format
    if dicom_time_str:
        hours = int(dicom_time_str[0:2])
        minutes = int(dicom_time_str[2:4])
        seconds = float(dicom_time_str[4:])
        return timedelta(hours=hours, minutes=minutes, seconds=seconds)
    return None

def extract_metadata(pet_dicom_file_path, ct_dicom_file_path):
    pet_dicom_data = pydicom.dcmread(pet_dicom_file_path)
    ct_dicom_data = pydicom.dcmread(ct_dicom_file_path)

    metadata = {
        #common
        'patient_id': get_data_element_value(pet_dicom_data.PatientID),
        'weight': get_data_element_value(pet_dicom_data.PatientWeight),
        'series_date': get_data_element_value(pet_dicom_data.SeriesDate),
        'series_time': get_data_element_value(pet_dicom_data.SeriesTime),
        'acquisition_date': get_data_element_value(pet_dicom_data.AcquisitionDate),
        'acquisition_time': get_data_element_value(pet_dicom_data.AcquisitionTime),


        #PET specific
        'rescale_slope_pet': get_data_element_value(pet_dicom_data.get((0x0028, 0x1053))),
        'rescale_intercept_pet': get_data_element_value(pet_dicom_data.get((0x0028, 0x1052))),
        'acquisition_time_pet': get_data_element_value(pet_dicom_data.AcquisitionTime),


        #CT specific
        'rescale_slope_ct': get_data_element_value(ct_dicom_data.get((0x0019, 0x1092))),
        'rescale_intercept_ct': get_data_element_value(ct_dicom_data.get((0x0019, 0x1093))),
        'acquisition_time_ct': get_data_element_value(ct_dicom_data.AcquisitionTime),
    }

    metadata['series_time'] = dicom_time_to_timedelta(metadata['series_time'])
    metadata['acquisition_time'] = dicom_time_to_timedelta(metadata['acquisition_time'])
    #metadata['acquisition_time_pet'] = dicom_time_to_timedelta(metadata['acquisition_time_pet'])
    #metadata['acquisition_time_ct'] = dicom_time_to_timedelta(metadata['acquisition_time_ct'])

    # Extract Radionuclide Information from the sequence
    rad_info_sequence = pet_dicom_data.get((0x0054, 0x0016), None)
    if rad_info_sequence:
        rad_info_item = rad_info_sequence[0]
        metadata['Radiopharmaceutical_start_time'] = get_data_element_value(rad_info_item.get((0x0018, 0x1072)))
        metadata['radionuclide_total_dose'] = get_data_element_value(rad_info_item.get((0x0018, 0x1074)))
        metadata['radionuclide_half_life'] = get_data_element_value(rad_info_item.get((0x0018, 0x1075)))

    return metadata

# Read DICOM metadata
metadata = extract_metadata(pet_dicom_file_path, ct_dicom_file_path)

print(metadata)


from datetime import datetime

# Convert DICOM time string to Python datetime object
def convert_dicom_time_to_datetime(dicom_time_str):
    return datetime.strptime(dicom_time_str, '%H%M%S.%f')

# Convert a timedelta to DICOM time format
def convert_timedelta_to_dicom_time(delta):
    # Extract hours, minutes, seconds and microseconds
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    microseconds = delta.microseconds
    # Construct a time string
    return f'{hours:02d}{minutes:02d}{seconds:02d}.{microseconds:06d}'


{'patient_id': 'PETCT_0117d7f11f', 'weight': '65.0', 'series_date': '20010913', 'series_time': datetime.timedelta(seconds=31322), 'acquisition_date': '20010913', 'acquisition_time': datetime.timedelta(seconds=31957, microseconds=29), 'rescale_slope_pet': '1.77409', 'rescale_intercept_pet': '0.0', 'acquisition_time_pet': '085237.000029', 'rescale_slope_ct': '0.9236', 'rescale_intercept_ct': '-1.6565', 'acquisition_time_ct': '083125.539000', 'Radiopharmaceutical_start_time': '074700.000000', 'radionuclide_total_dose': '313000000.0', 'radionuclide_half_life': '6586.2'}


4. Main functions to calculate the statistics:

In [128]:
def calculate_hu_metrics(ct_nifti_path, segmented_nifti_path, rescale_slope_ct, rescale_intercept_ct):
    # Load CT NIfTI file
    ct_img = nib.load(ct_nifti_path)
    ct_data = ct_img.get_fdata()

    # Load segmented NIfTI file for HU calculation and mask metrics
    segmented_img = nib.load(segmented_nifti_path)
    segmented_data = segmented_img.get_fdata()

    # Calculate HU values for the organ region
    hu_values_organ = (ct_data * segmented_data * rescale_slope_ct) + rescale_intercept_ct

    # Initialize variables with default values
    hu_mean_organ, hu_std_organ, mask_volume_ml = np.nan, np.nan, np.nan

    # Check if there are valid data points in the segmented region
    valid_data_points = hu_values_organ[segmented_data > 0]

    if valid_data_points.size > 0:
        # Calculate HU mean and standard deviation for the organ
        hu_mean_organ = valid_data_points.mean()
        hu_std_organ = valid_data_points.std()

        print("HU Mean:")
        print(hu_mean_organ)

        print("HU Standard Deviation:")
        print(hu_std_organ)

    else:
        print("No valid data points in the segmented organ region. Unable to calculate HU metrics.")

    # Calculate mask volume in milliliters
    voxel_volume_mm3 = np.prod(ct_img.header.get_zooms())  # Voxel volume in mm^3
    mask_volume_ml = np.sum(segmented_data) * voxel_volume_mm3 / 1000.0  # Convert to milliliters

    print("Mask Volume (ml):")
    print(mask_volume_ml)

    # finding non-zero voxels at the border of segmented volumes
    non_zero_voxels = int(
        (segmented_data[0, :, :].any() or segmented_data[-1, :, :].any() or
         segmented_data[:, 0, :].any() or segmented_data[:, -1, :].any() or
         segmented_data[:, :, 0].any() or segmented_data[:, :, -1].any())
    )

    print("Non-zero voxel:", non_zero_voxels)

    return hu_mean_organ, hu_std_organ, mask_volume_ml, non_zero_voxels

def calculate_organ_suv(nifti_file_path, segmented_nifti_path, weight, radionuclide_half_life, series_time, acquisition_time, radionuclide_total_dose, rescale_slope_pet, rescale_intercept_pet):
    # Load PET NIfTI file
    pet_img = nib.load(nifti_file_path)
    pet_data = pet_img.get_fdata()

    # Apply a median filter to reduce noise
    filtered_pet_data = median_filter(pet_data, size=5)  # The size parameter may need adjustment

    # Load segmented NIfTI file
    segmented_img = nib.load(segmented_nifti_path)
    segmented_data = segmented_img.get_fdata()

    # Calculate decayed dose
    decay_time = (acquisition_time - series_time).total_seconds()
    decayed_dose = radionuclide_total_dose * (2 ** (-decay_time / radionuclide_half_life))

    # Calculate SUVbw scale factor
    suvbw_scale_factor = (weight * 1000) / decayed_dose

    # Use the filtered PET data for SUV calculations
    suvbw_organ = (filtered_pet_data * segmented_data * rescale_slope_pet + rescale_intercept_pet) * suvbw_scale_factor

    # Initialize variables with default values
    mean_suvbw_organ, suv_peak_organ, suv_max_organ, suv_std_organ = np.nan, np.nan, np.nan, np.nan

    # Check if there are valid data points in the segmented region
    valid_data_points = suvbw_organ[segmented_data > 0]

    if valid_data_points.size > 0:
        # Calculate the mean SUVbw value for the organ
        mean_suvbw_organ = valid_data_points.mean()
        print("Mean SUVbw:")
        print(mean_suvbw_organ)

        # Calculate SUV Max for the organ
        suv_max_organ = valid_data_points.max()
        print("SUV Max:")
        print(suv_max_organ)

        # Calculate SUV Standard Deviation for the organ
        suv_std_organ = valid_data_points.std()
        print("SUV Standard Deviation:")
        print(suv_std_organ)

        # Find the spatial coordinates (indices) of the maximum SUV voxel
        max_suv_index = np.unravel_index(np.argmax(suvbw_organ), suvbw_organ.shape)

        # Define a small region (1 cm^3 sphere) around the maximum SUV voxel
        sphere_radius = int(np.ceil(1 / pet_img.header.get_zooms()[0]))  # in voxel units
        region_around_max_suv = suvbw_organ[
            max(0, max_suv_index[0] - sphere_radius):min(suvbw_organ.shape[0], max_suv_index[0] + sphere_radius + 1),
            max(0, max_suv_index[1] - sphere_radius):min(suvbw_organ.shape[1], max_suv_index[1] + sphere_radius + 1),
            max(0, max_suv_index[2] - sphere_radius):min(suvbw_organ.shape[2], max_suv_index[2] + sphere_radius + 1)
        ]

        # Calculate SUV Peak for the organ (average value within the 1-cm^3 sphere)
        # Calculate SUV Peak for the organ using median (more robust to noise)
        suv_peak_organ = np.median(region_around_max_suv[region_around_max_suv > 0])
        #suv_peak_organ = region_around_max_suv.mean()
        print("SUV Peak:")
        print(suv_peak_organ)

    else:
        print("No valid data points in the segmented organ region. Unable to calculate SUV metrics.")

    return mean_suvbw_organ, suv_peak_organ, suv_max_organ, suv_std_organ

def calculate_organ_sur(nifti_file_path, segmented_nifti_path, aorta_activity, rescale_slope_pet, rescale_intercept_pet):
    # Load PET NIfTI file
    pet_img = nib.load(nifti_file_path)
    pet_data = pet_img.get_fdata()

    # Apply a median filter to reduce noise
    filtered_pet_data = median_filter(pet_data, size=5)

    # Load segmented NIfTI file
    segmented_img = nib.load(segmented_nifti_path)
    segmented_data = segmented_img.get_fdata()

    # Calculate SUR by normalizing PET data with the aorta activity
    sur_organ = ((filtered_pet_data * segmented_data * rescale_slope_pet) + rescale_intercept_pet) / aorta_activity

    # Initialize variables
    mean_sur_organ, sur_peak_organ, sur_max_organ, sur_std_organ = np.nan, np.nan, np.nan, np.nan

    # Check for valid data points in the segmented region
    valid_data_points = sur_organ[segmented_data > 0]

    if valid_data_points.size > 0:
        mean_sur_organ = valid_data_points.mean()
        sur_max_organ = valid_data_points.max()
        sur_std_organ = valid_data_points.std()

        # Calculate SUR Peak
        max_sur_index = np.unravel_index(np.argmax(sur_organ), sur_organ.shape)
        sphere_radius = int(np.ceil(1 / pet_img.header.get_zooms()[0]))
        region_around_max_sur = sur_organ[
            max(0, max_sur_index[0] - sphere_radius):min(sur_organ.shape[0], max_sur_index[0] + sphere_radius + 1),
            max(0, max_sur_index[1] - sphere_radius):min(sur_organ.shape[1], max_sur_index[1] + sphere_radius + 1),
            max(0, max_sur_index[2] - sphere_radius):min(sur_organ.shape[2], max_sur_index[2] + sphere_radius + 1)
        ]
        sur_peak_organ = np.median(region_around_max_sur[region_around_max_sur > 0])

    return mean_sur_organ, sur_peak_organ, sur_max_organ, sur_std_organ




5. Output storage path: (Simply specify a path to your folder of interest. An excel sheet will be generated and all the outputs will be stored there.)

In [129]:
existing_excel_file_path = f"C:/Personal/ABX/patient_1/Results/{metadata['patient_id']}_08_02_24_patient1.xlsx"

6. Main code:

In [130]:

#Get a list of all NIfTI files in the "segmented_organs" folder
segmented_nifti_paths = [os.path.join(segmented_organs_folder, file) for file in os.listdir(segmented_organs_folder) if file.endswith(".nii.gz")]

# Initialize an empty DataFrame
existing_df = pd.DataFrame(columns=[
    'Patient ID', 'Inj Start [HHMMSS.FFFFFF]', 'CT Start [HHMMSS.FFFFFF]', 'PET Start [HHMMSS.FFFFFF]', 'Inj-PET [HHMMSS.FFFFFF]', 'CT-PET [HHMMSS.FFFFFF]', 'Organ', 'HU Mean', 'HU Std Dev', 'SUV Mean', 'SUV Std Dev', 'SUV Peak', 'SUV Max', 'SUR Mean', 'SUR Std Dev', 'SUR Peak', 'SUR Max', 'Non-zero Voxel', 'Mask Volume (ml)'
])
# Convert time strings to datetime objects
radiopharmaceutical_start_datetime = convert_dicom_time_to_datetime(metadata['Radiopharmaceutical_start_time'])
acquisition_time_pet_datetime = convert_dicom_time_to_datetime(metadata['acquisition_time_pet'])
acquisition_time_ct_datetime = convert_dicom_time_to_datetime(metadata['acquisition_time_ct'])

Injection_to_pet = acquisition_time_pet_datetime - radiopharmaceutical_start_datetime
ct_to_pet = acquisition_time_pet_datetime - acquisition_time_ct_datetime

# Convert the time difference to DICOM time format
Injection_to_pet_time_diff = convert_timedelta_to_dicom_time(Injection_to_pet)
ct_to_pet_time_diff = convert_timedelta_to_dicom_time(ct_to_pet)

print(Injection_to_pet_time_diff)
print(ct_to_pet_time_diff)


# Iterate through segmented organ NIfTI files
for segmented_nifti_path in segmented_nifti_paths:

    # Calculate HU metrics
    hu_mean, hu_std, mask_volume_ml, non_zero_voxels = calculate_hu_metrics(nifti_file_path_ct, segmented_nifti_path, metadata['rescale_slope_ct'], metadata['rescale_intercept_ct'])

    # Calculate SUV metrics
    #pet_weight = 65.0  # Assuming a constant weight for debuging
    #pet_radionuclide_half_life = 6586.2  # Assuming a constant half-life for debugging
    pet_mean_suvbw, pet_suv_peak, pet_suv_max, pet_suv_std = calculate_organ_suv(nifti_file_path_pet, segmented_nifti_path, metadata['weight'], metadata['radionuclide_half_life'], metadata['series_time'], metadata['acquisition_time'], metadata['radionuclide_total_dose'], metadata['rescale_slope_pet'], metadata['rescale_intercept_pet'])

    # Calculate SUR metrics
    pet_mean_sur, pet_sur_peak, pet_sur_max, pet_sur_std = calculate_organ_sur(nifti_file_path_pet, segmented_nifti_path, aorta_activity, metadata['rescale_slope_pet'], metadata['rescale_intercept_pet'])

    # Extract organ name from the segmented nifti file name
    organ_name = os.path.splitext(os.path.splitext(os.path.basename(segmented_nifti_path))[0])[0]

    #Injection_to_pet_time_diff = acquisition_time_pet_datetime - radiopharmaceutical_start_datetime
    #ct_to_pet_time_diff = acquisition_time_ct_datetime - acquisition_time_ct_datetime

    # Extract patient ID from nifti_file_path
    match = re.search(r'/NIFTI/(.*?)/', nifti_file_path_ct)
    if match:
        patient_id = match.group(1)
    else:
        print("Unable to extract Patient ID from nifti_file_path.")

    # Determine the index of the next empty row
    next_index = len(existing_df) + 1

    # Append the new results to the existing DataFrame with the calculated index
    new_data = {
        'Patient ID': [patient_id],

        'Inj Start [HHMMSS.FFFFFF]' : [metadata['Radiopharmaceutical_start_time']],
        'CT Start [HHMMSS.FFFFFF]': [metadata['acquisition_time_ct']], #metadata['rescale_slope_ct']
        'PET Start [HHMMSS.FFFFFF]': [metadata['acquisition_time_pet']], 
        'Inj-PET [HHMMSS.FFFFFF]': [Injection_to_pet_time_diff],
        'CT-PET [HHMMSS.FFFFFF]': [ct_to_pet_time_diff],

        'Organ': [organ_name],
        'HU Mean': [hu_mean],
        'HU Std Dev': [hu_std],
        'SUV Mean': [pet_mean_suvbw],
        'SUV Std Dev': [pet_suv_std],
        'SUV Peak': [pet_suv_peak],
        'SUV Max': [pet_suv_max],
        'SUR Mean': [pet_mean_sur],
        'SUR Std Dev': [pet_sur_std],
        'SUR Peak' : [pet_sur_peak],
        'SUR Max' : [pet_sur_max],
        'Non-zero Voxel' : [non_zero_voxels],
        'Mask Volume (ml)': [mask_volume_ml],
    }

    new_df = pd.DataFrame(new_data)
    new_df.index = [next_index]

    existing_df = pd.concat([existing_df, new_df], ignore_index=False)

# Save the updated DataFrame to the Excel file without the index column
existing_df.to_excel(existing_excel_file_path, index=False)

# Print a message indicating successful export
print(f"PET-CT Statistics: {existing_excel_file_path}")

010537.000029
002111.461029
HU Mean:
69.71785781821505
HU Standard Deviation:
34.52569762546819
Mask Volume (ml):
3.7323062896728514
Non-zero voxel: 0
Mean SUVbw:
2.313970935087981
SUV Max:
2.796639401744264
SUV Standard Deviation:
0.20839577766513648
SUV Peak:
2.660894898955162


C:\Users\Kathir\AppData\Local\Temp\ipykernel_10996\844011967.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  existing_df = pd.concat([existing_df, new_df], ignore_index=False)


HU Mean:
131.717836332247
HU Standard Deviation:
30.339087703876437
Mask Volume (ml):
195.4359983482361
Non-zero voxel: 0
Mean SUVbw:
2.7268283675296376
SUV Max:
4.747711508818379
SUV Standard Deviation:
0.2719037868685261
SUV Peak:
4.2118424458353925
HU Mean:
48.36886761773828
HU Standard Deviation:
27.179416912395066
Mask Volume (ml):
1124.1582134284972
Non-zero voxel: 1
Mean SUVbw:
7.748612786710046
SUV Max:
14.469652698616182
SUV Standard Deviation:
2.140266620054144
SUV Peak:
12.233801470532763
PET-CT Statistics: C:/Personal/ABX/patient_2/Results/PETCT_0117d7f11f_08_02_24_patient1.xlsx
